In [7]:
import requests
from typing import Dict, List
import os
from dotenv import load_dotenv
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
import time

load_dotenv()

class OAuthHandler:
    def __init__(self, client_id: str, client_secret: str, token_url: str):
        self.client_id = client_id
        self.client_secret = client_secret
        self.token_url = token_url
        self.token = None
        self.expires_at = 0

    def get_token(self):
        if self.token is None or time.time() > self.expires_at:
            client = BackendApplicationClient(client_id=self.client_id)
            oauth = OAuth2Session(client=client)
            self.token = oauth.fetch_token(
                token_url=self.token_url,
                client_id=self.client_id,
                client_secret=self.client_secret
            )
            self.expires_at = time.time() + self.token['expires_in'] - 300  # Refresh 5 minutes before expiry
        return self.token['access_token']

class Watch:
    def __init__(self, brand: str, model: str, price: float, description: str, condition: str, image_paths: List[str]):
        self.brand = brand
        self.model = model
        self.price = price
        self.description = description
        self.condition = condition
        self.image_paths = image_paths

class Marketplace:
    def __init__(self, name: str):
        self.name = name
        self.oauth_handler = self.setup_oauth()

    def setup_oauth(self) -> OAuthHandler:
        client_id = os.getenv(f"{self.name.upper()}_CLIENT_ID")
        client_secret = os.getenv(f"{self.name.upper()}_CLIENT_SECRET")
        token_url = os.getenv(f"{self.name.upper()}_TOKEN_URL")
        return OAuthHandler(client_id, client_secret, token_url)

    def get_headers(self):
        return {
            "Authorization": f"Bearer {self.oauth_handler.get_token()}",
            "Content-Type": "application/json"
        }

    def create_listing(self, watch: Watch) -> Dict:
        pass

    def upload_image(self, image_path: str) -> str:
        pass

class EbayMarketplace(Marketplace):
    def upload_image(self, image_path: str) -> str:
        url = "https://api.ebay.com/v1/item/uploadImage"
        
        headers = self.get_headers()
        headers["Content-Type"] = "image/jpeg"  # Adjust based on image type
        
        with open(image_path, 'rb') as image_file:
            response = requests.post(url, headers=headers, data=image_file)
        
        if response.status_code == 200:
            return response.json()['imageUrl']
        else:
            raise Exception(f"Failed to upload image: {response.text}")

    def create_listing(self, watch: Watch) -> Dict:
        image_urls = [self.upload_image(path) for path in watch.image_paths]
        
        url = "https://api.ebay.com/sell/inventory/v1/inventory_item"
        
        payload = {
            "product": {
                "title": f"{watch.brand} {watch.model}",
                "description": watch.description,
                "aspects": {
                    "Brand": [watch.brand],
                    "Model": [watch.model],
                    "Condition": [watch.condition]
                },
                "imageUrls": image_urls
            },
            "availability": {
                "shipToLocationAvailability": {
                    "quantity": 1
                }
            },
            "condition": watch.condition,
            "price": {
                "value": str(watch.price),
                "currency": "USD"
            }
        }
        
        response = requests.post(url, json=payload, headers=self.get_headers())
        return response.json()

class VintedMarketplace(Marketplace):
    def upload_image(self, image_path: str) -> str:
        url = "https://api.vinted.com/v1/images"
        
        with open(image_path, 'rb') as image_file:
            files = {'image': image_file}
            response = requests.post(url, headers=self.get_headers(), files=files)
        
        if response.status_code == 200:
            return response.json()['id']
        else:
            raise Exception(f"Failed to upload image: {response.text}")

    def create_listing(self, watch: Watch) -> Dict:
        image_ids = [self.upload_image(path) for path in watch.image_paths]
        
        url = "https://api.vinted.com/v1/items"
        
        payload = {
            "item": {
                "title": f"{watch.brand} {watch.model}",
                "description": watch.description,
                "price": str(watch.price),
                "brand_id": self.get_brand_id(watch.brand),
                "size_id": self.get_size_id("OneSize"),
                "status_id": self.get_status_id(watch.condition),
                "photo_ids": image_ids
            }
        }
        
        response = requests.post(url, json=payload, headers=self.get_headers())
        return response.json()

    # ... (other methods remain the same)

def create_watch_listing(watch: Watch, marketplaces: List[Marketplace]) -> Dict[str, Dict]:
    results = {}
    for marketplace in marketplaces:
        try:
            result = marketplace.create_listing(watch)
            results[marketplace.name] = result
        except Exception as e:
            results[marketplace.name] = {"error": str(e)}
    return results

def main():
    ebay = EbayMarketplace("eBay")
    vinted = VintedMarketplace("Vinted")
    marketplaces = [ebay, vinted]

    watch = Watch(
        brand="Rolex",
        model="Submariner",
        price=10000.00,
        description="Excellent condition Rolex Submariner. Barely worn, includes original box and papers.",
        condition="Used - Excellent",
        image_paths=["path/to/image1.jpg", "path/to/image2.jpg"]
    )

    results = create_watch_listing(watch, marketplaces)

    for marketplace, result in results.items():
        print(f"{marketplace}: {result}")

if __name__ == "__main__":
    main()

eBay: {'error': '(invalid_client) client authentication failed'}
Vinted: {'error': "[Errno 2] No such file or directory: 'path/to/image1.jpg'"}
